In [1]:
import pandas as pd
import numpy as np

In [13]:
url="https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
covidCounties = pd.read_csv(url, parse_dates=True, keep_default_na=False)

In [14]:
covidCounties = covidCounties[covidCounties['state'] !='Puerto Rico']

In [15]:
covidCounties['deaths'] = covidCounties['deaths'].str.strip().astype('float').astype('Int64')

In [16]:
covidCounties['date'] = pd.to_datetime(covidCounties['date'])
covidCounties.loc[covidCounties['county'] == 'New York City', 'fips']  = '36061'

In [17]:
covidCounties

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0
...,...,...,...,...,...,...
1498302,2021-07-08,Sweetwater,Wyoming,56037,4820,40
1498303,2021-07-08,Teton,Wyoming,56039,3807,11
1498304,2021-07-08,Uinta,Wyoming,56041,2328,13
1498305,2021-07-08,Washakie,Wyoming,56043,928,26


In [39]:
# Importing data
url="https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv"
countiesData = pd.read_csv(url, parse_dates=True, keep_default_na=False, encoding='ISO-8859-1')

In [40]:
countiesData = countiesData[countiesData['SUMLEV'] == 50][['SUMLEV','STATE','COUNTY','STNAME','CTYNAME','POPESTIMATE2019']]
countiesData.rename(columns={"POPESTIMATE2019": "population_2019"},  inplace = True)

In [41]:
countiesData['COUNTY'] = countiesData['COUNTY'].astype(str).str.zfill(3)
countiesData['STATE'] = countiesData['STATE'].astype(str).str.zfill(2)

In [42]:
countiesData['fips'] =  countiesData['STATE'] +  countiesData['COUNTY']
countiesData.dtypes

SUMLEV              int64
STATE              object
COUNTY             object
STNAME             object
CTYNAME            object
population_2019     int64
fips               object
dtype: object

In [43]:
joined = covidCounties.merge(countiesData, how='left', on='fips')
joined['population_2019'] = joined['population_2019'].astype('Int64')
joined['deaths'] = joined['deaths'].astype('Int64')

In [23]:
cty = joined[['county','CTYNAME']].drop_duplicates()
cty['county'] = cty['county'] + ' County' 
cty[cty['CTYNAME']  != cty['CTYNAME'] ]

,county,CTYNAME
418,Unknown County,NaN
5634,Kansas City County,NaN
37793,St. Croix County,NaN
37794,St. John County,NaN
37795,St. Thomas County,NaN
159131,Bristol Bay plus Lake and Peninsula County,NaN
268820,Joplin County,NaN
327916,Saipan County,NaN
327917,Tinian County,NaN
357201,Yakutat plus Hoonah-Angoon County,NaN


In [24]:
cty = joined[['county','state','CTYNAME']].drop_duplicates()
cty['county'] = cty['county'] + ' County' 
cty[cty['CTYNAME']  != cty['CTYNAME'] ]
# 61 counties without population

,county,state,CTYNAME
418,Unknown County,Rhode Island,NaN
1511,Unknown County,New Jersey,NaN
2265,Unknown County,Virgin Islands,NaN
2420,Unknown County,Guam,NaN
2926,Unknown County,Maine,NaN
...,...,...,...
327917,Tinian County,Northern Mariana Islands,NaN
357201,Yakutat plus Hoonah-Angoon County,Alaska,NaN
400232,Unknown County,South Carolina,NaN
763450,Unknown County,Oregon,NaN


In [45]:
joined = joined[['date','fips','county','state','cases','deaths','population_2019']]
joined.sort_values(['date']).reset_index(drop=True)
joined["comulative_cases"] = joined.groupby(['fips'])['cases'].cumsum(axis=0)
joined["comulative_deaths"] = joined.groupby(['fips'])['deaths'].cumsum(axis=0)

In [47]:
joined["comulative_deaths"] =joined["comulative_deaths"].astype('Int64')

joined.dtypes

date                 datetime64[ns]
fips                         object
county                       object
state                        object
cases                         int64
deaths                        Int64
population_2019               Int64
comulative_cases              int64
comulative_deaths             Int64
dtype: object